In [2]:
import sys
import os

# Get the current working directory
current_dir = os.getcwd()

# Add the current directory to the Python path
if current_dir not in sys.path:
    sys.path.append(current_dir)

# Verify the path
print(sys.path)

['/home/gautom/Documents/lavis', '/home/gautom/anaconda3/envs/lavis/lib/python38.zip', '/home/gautom/anaconda3/envs/lavis/lib/python3.8', '/home/gautom/anaconda3/envs/lavis/lib/python3.8/lib-dynload', '', '/home/gautom/.local/lib/python3.8/site-packages', '/home/gautom/anaconda3/envs/lavis/lib/python3.8/site-packages']


In [2]:
print(sys.path)

['/home/gautom/Documents/lavis', '/home/gautom/anaconda3/envs/lavis/lib/python38.zip', '/home/gautom/anaconda3/envs/lavis/lib/python3.8', '/home/gautom/anaconda3/envs/lavis/lib/python3.8/lib-dynload', '', '/home/gautom/.local/lib/python3.8/site-packages', '/home/gautom/anaconda3/envs/lavis/lib/python3.8/site-packages']


In [4]:
from pycocoevalcap.tokenizer.ptbtokenizer import PTBTokenizer

In [6]:
# Create an instance of PTBTokenizer
tokenizer = PTBTokenizer()

# Prepare your captions
captions_for_image = {
    0: [{'caption': 'man plays guitar'}],
    1: [{'caption': 'guitar being played by man'}, {'caption': 'man holds guitar'}]
}
# Use the tokenize function
tokenized_captions = tokenizer.tokenize(captions_for_image)

print(tokenized_captions)

{0: ['man plays guitar'], 1: ['guitar being played by man', 'man holds guitar']}


PTBTokenizer tokenized 13 tokens at 171.25 tokens per second.


In [13]:
from pycocoevalcap.tokenizer.ptbtokenizer import PTBTokenizer
from pycocoevalcap.cider.cider import Cider

class SimpleCIDErEval:
    def __init__(self):
        self.tokenizer = PTBTokenizer()
        self.cider_scorer = Cider()

    def evaluate(self, predictions, references):
        # Format the input for the tokenizer
        gts = {i: [{'caption': c} for c in refs] for i, refs in enumerate(references)}
        res = {i: [{'caption': p}] for i, p in enumerate(predictions)}

        # Tokenize
        gts_tokenized = self.tokenizer.tokenize(gts)
        res_tokenized = self.tokenizer.tokenize(res)

        # Compute CIDEr score
        score, scores = self.cider_scorer.compute_score(gts_tokenized, res_tokenized)

        return score, scores

In [14]:
import json
from datasets import COCODataset

coco_dataset = COCODataset(ann_file='./data/coco/annotations/captions_val2017.json',
                           img_dir='./data/coco/val2017')

f = open('coco_results.json')
results = json.load(f)
f.close()

candidates = [result['caption'] for result in results]
references = [coco_dataset.get_captions(result['image_id']) for result in results]

loading annotations into memory...
Done (t=0.05s)
creating index...
index created!


In [15]:
# Create evaluator
evaluator = SimpleCIDErEval()

overall_score, individual_scores = evaluator.evaluate(candidates, references)

print(f"Overall CIDEr score: {overall_score}")
print("Individual CIDEr scores:")
for i, (preds, refs, scores) in enumerate(zip(predictions, references, individual_scores)):
    print(f"\nImage {i}:")
    print(f"  Predictions: {preds}")
    print(f"  References: {refs}")
    print(f"  Scores: {scores}")

PTBTokenizer tokenized 61766 tokens at 333686.27 tokens per second.
PTBTokenizer tokenized 9242 tokens at 88974.17 tokens per second.


Overall CIDEr score: 1.2852764152502318
Individual CIDEr scores:

Image 0:
  Predictions: a man playing guitar
  References: ['A man is in a kitchen making pizzas.', 'Man in apron standing on front of oven with pans and bakeware', 'A baker is working in the kitchen rolling dough.', 'A person standing by a stove in a kitchen.', 'A table with pies being made and a person standing near a wall with pots and pans hanging on the wall.']
  Scores: 0.5089577681230059

Image 1:
  Predictions: a woman singing
  References: ['The dining table near the kitchen has a bowl of fruit on it.', 'A small kitchen has various appliances and a table.', 'The kitchen is clean and ready for us to see.', 'A kitchen and dining area decorated in white.', 'A kitchen that has a bowl of fruit on the table.']
  Scores: 0.6427021227507621


In [12]:
from pycocoevalcap.tokenizer.ptbtokenizer import PTBTokenizer
from pycocoevalcap.cider.cider import Cider

class SimpleCIDErEval:
    def __init__(self):
        self.tokenizer = PTBTokenizer()
        self.cider_scorer = Cider()

    def evaluate(self, predictions, references):
        # Format the input for the tokenizer
        gts = {i: [{'caption': c} for c in refs] for i, refs in enumerate(references)}
        res = {i: [{'caption': p}] for i, p in enumerate(predictions)}

        # Tokenize
        gts_tokenized = self.tokenizer.tokenize(gts)
        res_tokenized = self.tokenizer.tokenize(res)

        # Compute CIDEr score
        score, scores = self.cider_scorer.compute_score(gts_tokenized, res_tokenized)

        return score, scores

# Example usage
if __name__ == "__main__":
    # Example data
    predictions = [
        "a man playing guitar",
        "a woman singing"
    ]
    references = [
        ["man plays guitar", "guitarist performing"],
        ["woman sings into microphone", "singer on stage", "vocal performance"]
    ]

    # Create evaluator
    evaluator = SimpleCIDErEval()

    # Compute scores
    overall_score, individual_scores = evaluator.evaluate(predictions, references)

    # Print results
    print(f"Overall CIDEr score: {overall_score}")
    print("Individual CIDEr scores:")
    for i, (pred, refs, score) in enumerate(zip(predictions, references, individual_scores)):
        print(f"\nImage {i}:")
        print(f"  Prediction: {pred}")
        print(f"  References: {refs}")
        print(f"  Score: {score}")

PTBTokenizer tokenized 18 tokens at 248.91 tokens per second.


Overall CIDEr score: 0.4744891202740206
Individual CIDEr scores:

Image 0:
  Prediction: a man playing guitar
  References: ['man plays guitar', 'guitarist performing']
  Score: 0.7117336804110309

Image 1:
  Prediction: a woman singing
  References: ['woman sings into microphone', 'singer on stage', 'vocal performance']
  Score: 0.2372445601370103


PTBTokenizer tokenized 8 tokens at 108.47 tokens per second.
